# Running a Federated Cycle with Synergos

In a federated learning system, there are many contributory participants, known as Worker nodes, which receive a global model to train on, with their own local dataset. The dataset does not leave the individual Worker nodes at any point, and remains private to the node.

The job to synchronize, orchestrate and initiate an federated learning cycle, falls on a Trusted Third Party (TTP). The TTP pushes out the global model architecture and parameters for the individual nodes to train on, calling upon the required data, based on tags, e.g "training", which points to relevant data on the individual nodes. At no point does the TTP receive, copy or access the Worker nodes' local datasets.

![Simulated Synergos Cluster Grid](../../docs/images/syncluster_setup.png "A simple Synergos Cluster setup")

This tutorial aims to give you an understanding of how to use the synergos package to run a full federated learning cycle on a `Synergos Cluster` grid. 

In a `Synergos Cluster` Grid, with the inclusion of a new director and queue component, you will be able to parallelize your jobs, where the number of concurrent jobs possible is equal to the number of sub-grids. This is done alongside all quality-of-life components supported in a `Synergos Plus` grid.

In this tutorial, you will go through the steps required by each participant (TTP and Worker), by simulating each of them locally with docker containers. Specifically, we will simulate a Director and 2 sub-grids, each of which has a TTP and 2 Workers, allowing us to perform 2 concurrent federated operations at any time.

At the end of this, we will have:
- Connected the participants
- Trained the model
- Evaluate the model


## About the Dataset and Task

The dataset used in this notebook is on leaf texture, comprising 64 predictor features, and 1 target feature for 16 classes. The dataset is available in the same directory as this notebook. Within the dataset directory, `data1` is for Worker 1 and `data2` is for Worker 2. The task to be carried out will be multi-classification.

The dataset we have provided is a processed subset of the [original One-hundred plant species leaves dataset](https://archive.ics.uci.edu/ml/datasets/One-hundred+plant+species+leaves+data+set).

**Reference:**
- *Dua, D. and Graff, C. (2019). UCI Machine Learning Repository [http://archive.ics.uci.edu/ml]. Irvine, CA: University of California, School of Information and Computer Science.*
- *Charles Mallah, James Cope, James Orwell. Plant Leaf Classification Using Probabilistic Integration of Shape, Texture and Margin Features. Signal Processing, Pattern Recognition and Applications, in press. 2013.*

## Initiating the docker containers

Before we begin, we have to start the docker containers.

### A. Initialization via `Synergos Simulator`

In `Synergos Simulator`, a sandboxed environment has been created for you!

By running:

    `docker-compose -f docker-compose-syncluster.yml up --build`
    
the following components will be started:
- Director
- Sub-Grid 1
  - TTP_1 (Cluster)
  - Worker_1_n1
  - Worker_2_n1
- Sub-Grid 2
  - TTP_2 (Cluster)
  - Worker_1_n2
  - Worker_2_n2
- Synergos UI
- Synergos Logger
- Synergos MLOps
- Synergos MQ

Refer to [this](https://github.com/aimakerspace/synergos_simulator) for all the pre-allocated host & port mappings. 

### B. Manual Initialization

Firstly, pull the required docker images with the following commands:

1. Synergos Director:
    
    `docker pull gcr.io/synergos-aisg/synergos_director:v0.1.0`

2. Synergos TTP (Cluster): 

    `docker pull gcr.io/synergos-aisg/synergos_ttp_cluster:v0.1.0`

3. Synergos Worker: 

    `docker pull gcr.io/synergos-aisg/synergos_worker:v0.1.0`
    
4. Synergos MLOps:

    `docker pull gcr.io/synergos-aisg/synergos_mlops:v0.1.0`
    
5. Synergos MQ:

    `docker pull gcr.io/synergos-aisg/synergos_mq:v0.1.0`

Next, in <u>separate</u> CLI terminals, run the following command(s):

**Note: For Windows users, it is advisable to use powershell or command prompt based interfaces**

#### Director

```
docker run --rm
    -p 5000:5000 
    -v <directory leaf_textures/orchestrator_data>:/orchestrator/data
    -v <directory leaf_textures/orchestrator_outputs>:/orchestrator/outputs
    -v <directory leaf_textures/mlflow>:/mlflow
    --name director 
    gcr.io/synergos-aisg/synergos_director:v0.1.0 
        --id ttp 
        --logging_variant graylog <IP Synergos Logger> <TTP port>
        --queue rabbitmq <IP Synergos Logger> <AMQP port>
```

#### Sub-Grid 1

- **TTP_1**
```
docker run --rm
    -p 6000:5000 
    -p 9020:8020
    -v <directory leaf_textures/orchestrator_data>:/orchestrator/data
    -v <directory leaf_textures/orchestrator_outputs>:/orchestrator/outputs
    --name ttp_1 
    gcr.io/synergos-aisg/synergos_ttp_cluster:v0.1.0 
        --id ttp 
        --logging_variant graylog <IP Synergos Logger> <TTP port>
        --queue rabbitmq <IP Synergos Logger> <AMQP port>
```

- **WORKER_1 Node 1**
```
docker run --rm
    -p 5001:5000 
    -p 8021:8020
    -v <directory leaf_textures/data1>:/worker/data 
    -v <directory leaf_textures/outputs_1>:/worker/outputs 
    --name worker_1_n1 
    gcr.io/synergos-aisg/synergos_worker:v0.1.0 
        --id worker_1_n1 
        --logging_variant graylog <IP Synergos Logger> <Worker port>
        --queue rabbitmq <IP Synergos Logger> <AMQP port>
```

- **WORKER_2 Node 1**
```
docker run --rm
    -p 5002:5000 
    -p 8022:8020
    -v <directory leaf_textures/data2>:/worker/data 
    -v <directory leaf_textures/outputs_2>:/worker/outputs 
    --name worker_2_n1
    gcr.io/synergos-aisg/synergos_worker:v0.1.0
        --id worker_2_n1 
        --logging_variant graylog <IP Synergos Logger> <Worker port>
        --queue rabbitmq <IP Synergos Logger> <AMQP port>
```

#### Sub-Grid 2

- **TTP_2**
```
docker run --rm
    -p 7000:5000 
    -p 10020:8020
    -v <directory leaf_textures/orchestrator_data>:/orchestrator/data
    -v <directory leaf_textures/orchestrator_outputs>:/orchestrator/outputs
    --name ttp_2 
    gcr.io/synergos-aisg/synergos_ttp_cluster:v0.1.0 
        --id ttp 
        --logging_variant graylog <IP Synergos Logger> <TTP port>
        --queue rabbitmq <IP Synergos Logger> <AMQP port>
```

- **WORKER_1 Node 2**
```
docker run --rm
    -p 5003:5000 
    -p 8023:8020
    -v <directory leaf_textures/data1>:/worker/data 
    -v <directory leaf_textures/outputs_1>:/worker/outputs 
    --name worker_1_n2 
    gcr.io/synergos-aisg/synergos_worker:v0.1.0 
        --id worker_1_n2 
        --logging_variant graylog <IP Synergos Logger> <Worker port>
        --queue rabbitmq <IP Synergos Logger> <AMQP port>
```

- **WORKER_2 Node 2**
```
docker run --rm
    -p 5004:5000 
    -p 8024:8020
    -v <directory leaf_textures/data2>:/worker/data 
    -v <directory leaf_textures/outputs_2>:/worker/outputs 
    --name worker_2_n2
    gcr.io/synergos-aisg/synergos_worker:v0.1.0
        --id worker_2_n2 
        --logging_variant graylog <IP Synergos Logger> <Worker port>
        --queue rabbitmq <IP Synergos Logger> <AMQP port>
```

#### Synergos MLOps

```
docker run --rm 
    -p 5500:5500 
    -v /path/to/mlflow_test/:/mlflow    # <-- IMPT! Same as orchestrator's
    --name synmlops 
    gcr.io/synergos-aisg/synergos_mlops:v0.1.0
```

#### Synergos MQ

```
docker run --rm         
    -p 15672:15672      # UI port
    -p 5672:5672        # AMQP port
    --name synergos_mq 
    gcr.io/synergos-aisg/synergos_mq:v0.1.0
```

#### Synergos UI

- Refer to these [instructions](https://github.com/aimakerspace/synergos_ui) to deploy `Synergos UI`.

#### Synergos Logger

- Refer to these [instructions](https://github.com/aimakerspace/synergos_logger) to deploy `Synergos Logger`.


Once ready, for each terminal, you should see a REST server running on http://0.0.0.0:5000 of the container.

You are now ready for the next step.

## Configurations

### A. Configuring `Synergos Simulator`

All hosts & ports have already been pre-allocated!

Refer to [this](https://github.com/aimakerspace/synergos_simulator) for all the pre-allocated host & port mappings.

### B. Configuring your manual setup

In a new terminal, run `docker inspect bridge` and find the IPv4Address for each container. Ideally, the containers should have the following addresses:
- director address: `172.17.0.2`
- Sub-Grid 1
  - ttp_1 address: `172.17.0.3`
  - worker_1_n1 address: `172.17.0.4`
  - worker_2_n1 address: `172.17.0.5`
- Sub-Grid 2
  - ttp_2 address: `172.17.0.6`
  - worker_1_n2 address: `172.17.0.7`
  - worker_2_n2 address: `172.17.0.8`
- UI address: `172.17.0.9`
- Logger address: `172.17.0.14`
- MLOps address: `172.17.0.15`
- MQ address: `172.17.0.16`

If not, just note the relevant IP addresses for each docker container.

Run the following cells below.

**Note: For Windows users, `host` should be Docker Desktop VM's IP. Follow [this](https://stackoverflow.com/questions/58073936/how-to-get-ip-address-of-docker-desktop-vm) on instructions to find IP**

In [1]:
from synergos import Driver

host = "172.20.0.2" # Different for Windows users
port = 5000

# Initiate Driver
driver = Driver(host=host, port=port)

## Phase 1: Registration

Submitting Orchestrator & Participant metadata

#### 1A. TTP creates a collaboration

In [ ]:
collab_task = driver.collaborations
collab_task.create('test_collaboration')

#### 1B. TTP controller creates a project

In [ ]:
driver.projects.create(
    collab_id="test_collaboration",
    project_id="test_project",
    action="classify",
    incentives={
        'tier_1': [],
        'tier_2': [],
    }
)

#### 1C. TTP controller creates an experiment

In [ ]:
driver.experiments.create(
    collab_id="test_collaboration",
    project_id="test_project",
    expt_id="test_experiment",
    model=[
        {
            "activation": "sigmoid",
            "is_input": True,
            "l_type": "Linear",
            "structure": {
                "bias": True,
                "in_features": 64,
                "out_features": 32
            }
        },
        {
            "activation": "softmax",
            "is_input": False,
            "l_type": "Linear",
            "structure": {
                "bias": True,
                "in_features": 32,
                "out_features": 16
            }
        },
    ]
)

#### 1D. TTP controller creates a run

In [ ]:
driver.runs.create(
    collab_id="test_collaboration",
    project_id="test_project",
    expt_id="test_experiment",
    run_id="test_run",
    rounds=2, 
    epochs=1,
    base_lr=0.0005,
    max_lr=0.005,
    criterion="NLLLoss"
)

#### 1E. Participants registers their servers' configurations and roles

In [ ]:
participant_resp_1 = driver.participants.create(
    participant_id="worker_1",
)

display(participant_resp_1)

participant_resp_2 = driver.participants.create(
    participant_id="worker_2",
)

display(participant_resp_2)

In [ ]:
registration_task = driver.registrations

# Add and register worker_1 node
registration_task.add_node(
    host='172.17.0.2',
    port=8020,
    f_port=5000,
    log_msgs=True,
    verbose=True
)
registration_task.list_nodes()

registration_task.create(
    collab_id="test_collaboration",
    project_id="test_project",
    participant_id="worker_1",
    role="host"
)

In [ ]:
registration_task = driver.registrations
registration_task.add_node(
    host='172.17.0.3',
    port=8020,
    f_port=5000,
    log_msgs=True,
    verbose=True
)
registration_task.list_nodes()

registration_task.create(
    collab_id="test_collaboration",
    project_id="test_project",
    participant_id="worker_2",
    role="guest"
)

#### 1F. Participants registers their tags for a specific project

In [ ]:
driver.tags.create(
    collab_id="test_collaboration",
    project_id="test_project",
    participant_id="worker_1",
    train=[["train"]],
    evaluate=[["evaluate"]]
)

driver.tags.create(
    collab_id="test_collaboration",
    project_id="test_project",
    participant_id="worker_2",
    train=[["train"]],
    evaluate=[["evaluate"]]
)

## Phase 2: 
Alignment, Training & Optimisation

#### 2A. Perform multiple feature alignment to dynamically configure datasets and models for cross-grid compatibility

In [ ]:
driver.alignments.create(collab_id='test_collaboration',
                         project_id="test_project",
                         verbose=False,
                         log_msg=False)

#### 2B. Trigger training across the federated grid

In [ ]:
model_resp = driver.models.create(
    collab_id="test_collaboration",
    project_id="test_project",
    expt_id="test_experiment",
    run_id="test_run",
    log_msg=False,
    verbose=False
)
display(model_resp)

## Phase 3: EVALUATE 
Validation & Predictions

#### 3A. Perform validation(s) of combination(s)

In [ ]:
driver.validations.create(
    collab_id='test_collaboration',
    project_id="test_project",
    expt_id="test_experiment",
    run_id="test_run",
    log_msg=False,
    verbose=False
)

#### 3B. Perform prediction(s) of combination(s)

In [ ]:
driver.predictions.create(
    collab_id="test_collaboration",
    tags={"test_project": [["predict"]]},
    participant_id="worker_1",
    project_id="test_project",
    expt_id="test_experiment",
    run_id="test_run"
)